In [1]:
import pandas as pd
from datetime import datetime

import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm, tqdm_notebook
pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge
fname = 'data.csv'

def init_data(fname):
    data = pd.read_csv('data.csv')
    data['yx_spread'] = data.yprice - data.xprice
    data['yx_relation'] = data.yprice / data.xprice
    data['xy_relation'] = data.xprice / data.yprice
    data['xy_geom'] = np.sqrt(data.xprice * data.yprice)
    data['xy_garmonic'] = 2 / (1 / data.xprice + 1 / data.yprice)
    
#     data.xprice = (data.xprice - data.xprice.min())# / data.xprice.std() 
#     data.yprice = (data.yprice - data.yprice.min())# / data.yprice.std() 
    data['timestamp'] = data['timestamp'] // 1000
    data['timestamp'] = data['timestamp'].apply(lambda stamp: datetime.fromtimestamp(stamp))
    data['timestamp'] = data['timestamp'] - pd.Timedelta(hours=1) # for flexibility
    data.index = data['timestamp']
    
    data['weekday'] = data.timestamp.dt.weekday
    data['day'] = (data.timestamp.dt.date - data.timestamp.dt.date.min()).apply(lambda x: int(x.days))
    day_close_time = data.day.map(data.groupby('day').timestamp.max())
    data['periods_before_closing'] = (day_close_time - data.timestamp).apply(lambda x: x.seconds // 10)
    day_open_time = data.day.map(data.groupby('day').timestamp.min())
    data['periods_after_opening'] = (data.timestamp - day_open_time).apply(lambda x: x.seconds // 10)
#     data.drop('timestamp', 1, inplace=True)
    return data
    
def time_split(data, valid_ratio, test_ratio):
    n_valid = max(1, int(data.shape[0] * valid_ratio))
    n_test = max(1, int(data.shape[0] * test_ratio))
    n_train = data.shape[0] - n_valid - n_test
    
    train = data.iloc[:n_train].reset_index(drop=True).copy()
    valid = data.iloc[n_train:-n_test].reset_index(drop=True).copy()
    test = data.iloc[-n_test:].reset_index(drop=True).copy()
    merged_test = valid.append(test).reset_index(drop=True)
    return train, valid, test

In [2]:
def add_diffs(df, column, uselags):
    new_columns = []
    for lag in uselags:
        colname = '{}_diff_{}'.format(column, lag)
        df.loc[:, colname] = df[column].diff(lag)
        new_columns.append(colname)
    print(new_columns)
    return new_columns

def add_shifts(df, column, uselags):
    new_columns = []
    for lag in uselags:
        colname = '{}_lag_{}'.format(column, lag)
        df.loc[:, colname] = df[column].shift(lag)
        new_columns.append(colname)
    print(new_columns)
    return new_columns

def add_rolling_mean(df, column, windows):
    new_columns = []
    for window_size in windows:
        colname = '{}_ma_{}'.format(column, window_size)
        df.loc[:, colname] = df[column].rolling(window=window_size).mean()
        new_columns.append(colname)
    print(new_columns)
    return new_columns

def add_curstom_rolling_operation(df, column, agg_function, function_name, windows):
    new_columns = []
    for window_size in windows:
        colname = '{}_{}_{}'.format(column, function_name, window_size)
        df.loc[:, colname] = df[column].rolling(window=window_size).agg(agg_function)
        new_columns.append(colname)
    print(new_columns)
    return new_columns  

def rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

def add_rsi(df, column, windows):
    new_columns = []
    for window_size in windows:
        colname = '{}_rsi_{}'.format(column, window_size)
        df.loc[:, colname] = rsiFunc(df[column].values, window_size)
        new_columns.append(colname)
    print(new_columns)
    return new_columns  

def add_ewma(df, column, windows):
    new_columns = []
    for window_size in windows:
        colname = '{}_ewma_{}'.format(column, window_size)
        df.loc[:, colname] = pd.Series.ewm(df[column], span=window_size).mean()
        new_columns.append(colname)
    print(new_columns)
    return new_columns 

def add_time_depended_rolling(df, source_column, windows, agg_fun, agg_repr):
    '''
        df: source dataframe
        source_column: column for building feature
        windows: list with periods (1 period = 10 sec)
        agg_fun: aggregation function
        agg_repr: name of agg function
    '''    
    new_cols = []
    for agg_period in windows:
        agg_shifts = range(10, agg_period * 10, 10)
        period_repr = '{}s'.format(agg_period * 10)
        
        agg_helper_df = df[source_column].resample(
            period_repr, label='right', closed='right').agg(agg_fun)
                                             
        for shift in agg_shifts:
            agg_helper_df = agg_helper_df.append(df[source_column].resample(
                period_repr, label='right', closed='right', base=shift).agg(agg_fun))
        colname = '{}_time_{}_{}'.format(source_column, agg_repr, agg_period)
        df.loc[:, colname] = agg_helper_df
        new_cols.append(colname)
    print(new_cols)
    return new_cols

In [3]:
def add_hand_feats(df):
    close_price_per_day = df.groupby('day').timestamp.max().shift(1).map(
        df[['timestamp', 'yprice']].set_index('timestamp').yprice)
    df.loc[:, 'ydiff_from_closing'] = (df.day.map(close_price_per_day) - df.yprice).fillna(0)
    close_price_per_day = df.groupby('day').timestamp.max().shift(1).map(
        df[['timestamp', 'xprice']].set_index('timestamp').xprice)
    df.loc[:, 'xdiff_from_closing'] = (df.day.map(close_price_per_day) - df.yprice).fillna(0)
    
    open_price_per_day = df.groupby('day').timestamp.min().map(
        df[['timestamp', 'yprice']].set_index('timestamp').yprice)
    df.loc[:, 'ydiff_from_opening'] = (df.day.map(open_price_per_day) - df.yprice)
    
    open_price_per_day = df.groupby('day').timestamp.min().map(
        df[['timestamp', 'xprice']].set_index('timestamp').xprice)
    df.loc[:, 'xdiff_from_opening'] = (df.day.map(open_price_per_day) - df.xprice)
    new_columns = ['ydiff_from_closing', 'xdiff_from_closing', 'ydiff_from_opening', 'xdiff_from_opening']
    print(new_columns)
    return new_columns

def add_full_history_diff(df, col):
    mean = df[col].cumsum() / np.arange(1, df.shape[0] + 1)
    new_col = '{}_full_history_diff'.format(col)
    df.loc[:, new_col] = df[col] - mean
    print(new_col)
    return new_col

In [4]:
def validate_sklearn_model(model, data, selected_cols, valid_ratio, test_ratio, droprows=0, 
                           verbose=True, only_valid=False):
    train, valid, test = time_split(data, valid_ratio, test_ratio)
    train.drop(np.arange(droprows), inplace=True)
    train.dropna(inplace=True)
    if verbose:
        print('Data shapes: ', train.shape, valid.shape, test.shape)

    metrics_dict = {}
    
    if valid_ratio!=0:
        model.fit(train[selected_cols], train.returns)
        y_valid_predicted = model.predict(valid[selected_cols])
        y_valid_predicted[valid.periods_before_closing == 0] = 0

        metrics_dict['valid_mse'] = mean_squared_error(y_valid_predicted, valid.returns)
        metrics_dict['valid_r2'] = r2_score(valid.returns, y_valid_predicted) * 100
        if verbose:
            print('\nValid MSE: \t\t {:.5}'.format(metrics_dict['valid_mse']))
            print('Valid R2 (x100): \t {:.5}'.format(metrics_dict['valid_r2']))
    
    if not only_valid:
        model.fit(train.append(valid)[selected_cols], train.append(valid).returns)
        y_test_predicted = model.predict(test[selected_cols])
        y_test_predicted[test.periods_before_closing == 0] = 0

        metrics_dict['test_mse'] = mean_squared_error(y_test_predicted, test.returns)
        metrics_dict['test_r2'] = r2_score(test.returns, y_test_predicted) * 100
        if verbose:
            print('\nTest MSE: \t\t {:.5}'.format(metrics_dict['test_mse']))
            print('Test R2 (x100): \t {:.5}'.format(metrics_dict['test_r2']))
    
    return metrics_dict


def greedy_add_del_strategy(model, data, cols, valid_ratio, test_ratio, droprows=0, add_frequency=1):
    selected_cols = cols.copy()
    removed_cols = []
    current_step = 0
    
    current_score = -float('inf')
    
    while selected_cols:
        current_step += 1
        if current_step % add_frequency == 0:
            for col in removed_cols:
                current_cols = selected_cols + [col]
                current_metrics = validate_sklearn_model(
                    model, data, current_cols,
                    valid_ratio=valid_ratio, test_ratio=test_ratio, droprows=droprows,
                    verbose=False, only_valid=True
                )
                if current_metrics['valid_r2'] > current_score:
                    current_score = current_metrics['valid_r2']
                    selected_cols.append(col)
                    print('added {}: r2: {:.5}'.format(col, current_score))

        best_score_by_iter = -float('inf')
        worst_col = ''
        for col in selected_cols:
            current_cols = [c for c in selected_cols if c!=col]
            current_metrics = validate_sklearn_model(
                model, data, current_cols, 
                valid_ratio, test_ratio, droprows,
                verbose=False, only_valid=True
            )

            if current_metrics['valid_r2'] > best_score_by_iter:
                best_score_by_iter = current_metrics['valid_r2']
                worst_col = col
        if best_score_by_iter > current_score:
            current_score = best_score_by_iter
            print('removed {}: r2: {:.5}'.format(worst_col, best_score_by_iter))
            selected_cols.remove(worst_col)
            removed_cols.append(worst_col)
        else:
            return selected_cols
        
def greedy_add_strategy(model, data, base_cols, additional_cols, valid_ratio, test_ratio, droprows=0):
    current_score = validate_sklearn_model(
        model, data, base_cols,
        valid_ratio, test_ratio, droprows,
        verbose=False, only_valid=True
    )['valid_r2']
    is_continue_search = True
    while is_continue_search:
        is_continue_search = False
        for col in additional_cols:
            current_cols = base_cols + [col]
            current_metrics = validate_sklearn_model(
                model, data, current_cols,
                valid_ratio, test_ratio, droprows,
                verbose=False, only_valid=True
            )
            if current_metrics['valid_r2'] > current_score:
                current_score = current_metrics['valid_r2']
                base_cols.append(col)
                additional_cols.remove(col)
                is_continue_search = True
                print('added {}: r2: {:.5}'.format(col, current_score))
        
    return base_cols

In [5]:
short_agg_periods = [6, 60, 360]
oneday_agg_periods = [6, 60, 360, 720, 1410]
twoweeks_agg_periods = [6, 60, 360, 720, 1410, 2820, 7050, 14100]

month_day_lags = 1410 * np.arange(1, 20)

valid_ratio = 0.2
test_ratio = 0.15

- 6 - 1min
- 60 - 10min
- 360 - 1hour
- 1410 - 1workday (~ 4 hours per day)
- 7050 - 1workweek (5 days per week)
- 28200 - 1 workmonth (~ 4 weeks per month)

## Heap of features

In [11]:
usecols = [
    'xprice', 'yprice',
    'yx_relation', 'xy_relation',
    'yx_spread', 'xy_geom',
    'periods_before_closing'
]

data = init_data(fname)

hand_crafted_cols = add_hand_feats(data)
usecols.extend(hand_crafted_cols)

xcols = add_time_depended_rolling(data, 'xprice', short_agg_periods, np.mean, 'mean')
for col in xcols:
    data[col] = data.xprice - data[col]
usecols.extend(xcols)

ycols = add_time_depended_rolling(data, 'yprice', short_agg_periods, np.mean, 'mean')
for col in ycols:
    data[col] = data.yprice - data[col]
usecols.extend(ycols)

usecols.append(add_full_history_diff(data, 'xprice'))
usecols.append(add_full_history_diff(data, 'yprice'))
usecols.append(add_full_history_diff(data, 'yx_relation'))
usecols.append(add_full_history_diff(data, 'xy_geom'))

['ydiff_from_closing', 'xdiff_from_closing', 'ydiff_from_opening', 'xdiff_from_opening']
['xprice_time_mean_6', 'xprice_time_mean_60', 'xprice_time_mean_360']
['yprice_time_mean_6', 'yprice_time_mean_60', 'yprice_time_mean_360']
xprice_full_history_diff
yprice_full_history_diff
yx_relation_full_history_diff
xy_geom_full_history_diff


In [12]:
model = Ridge(alpha=10)
filtered_cols = greedy_add_del_strategy(model, data, usecols, valid_ratio, test_ratio,
                                        droprows=7050, add_frequency=4)
validate_sklearn_model(model, data, filtered_cols, valid_ratio, test_ratio, droprows=7050);

removed yprice_time_mean_60: r2: 0.84279
removed ydiff_from_opening: r2: 0.94374
removed xy_geom: r2: 1.0141
removed yprice_time_mean_6: r2: 1.0657
removed xy_relation: r2: 1.0659
Data shapes:  (215660, 27) (68526, 27) (51394, 27)

Valid MSE: 		 0.019382
Valid R2 (x100): 	 1.0659

Test MSE: 		 0.015794
Test R2 (x100): 	 0.76929


In [14]:
new_cols = add_rsi(data, 'yx_spread', twoweeks_agg_periods)
usecols.extend(new_cols)

model = Ridge(alpha=10)

selected_cols = greedy_add_strategy(model, data, filtered_cols, new_cols,
                                    valid_ratio, test_ratio, droprows=7050)
validate_sklearn_model(model, data, selected_cols, valid_ratio, test_ratio, droprows=7050);

['yx_spread_rsi_6', 'yx_spread_rsi_60', 'yx_spread_rsi_360', 'yx_spread_rsi_720', 'yx_spread_rsi_1410', 'yx_spread_rsi_2820', 'yx_spread_rsi_7050', 'yx_spread_rsi_14100']
added yx_spread_rsi_6: r2: 1.1793
added yx_spread_rsi_60: r2: 1.1793
Data shapes:  (215660, 35) (68526, 35) (51394, 35)

Valid MSE: 		 0.01936
Valid R2 (x100): 	 1.1793

Test MSE: 		 0.015835
Test R2 (x100): 	 0.51504


In [16]:
new_cols = add_rsi(data, 'yx_relation', twoweeks_agg_periods)
usecols.extend(new_cols)

model = Ridge(alpha=10)

selected_cols = greedy_add_strategy(model, data, selected_cols, new_cols,
                                    valid_ratio, test_ratio, droprows=7050)
validate_sklearn_model(model, data, selected_cols, valid_ratio, test_ratio, droprows=7050);

['yx_relation_rsi_6', 'yx_relation_rsi_60', 'yx_relation_rsi_360', 'yx_relation_rsi_720', 'yx_relation_rsi_1410', 'yx_relation_rsi_2820', 'yx_relation_rsi_7050', 'yx_relation_rsi_14100']
added yx_relation_rsi_60: r2: 1.2571
Data shapes:  (215660, 43) (68526, 43) (51394, 43)

Valid MSE: 		 0.019345
Valid R2 (x100): 	 1.2571

Test MSE: 		 0.015835
Test R2 (x100): 	 0.51512


In [18]:
new_cols = add_rsi(data, 'xy_relation', twoweeks_agg_periods)
usecols.extend(new_cols)

model = Ridge(alpha=10)

selected_cols = greedy_add_strategy(model, data, selected_cols, new_cols,
                                    valid_ratio, test_ratio, droprows=7050)
validate_sklearn_model(model, data, selected_cols, valid_ratio, test_ratio, droprows=7050);

['xy_relation_rsi_6', 'xy_relation_rsi_60', 'xy_relation_rsi_360', 'xy_relation_rsi_720', 'xy_relation_rsi_1410', 'xy_relation_rsi_2820', 'xy_relation_rsi_7050', 'xy_relation_rsi_14100']
added xy_relation_rsi_60: r2: 1.4096
Data shapes:  (215660, 51) (68526, 51) (51394, 51)

Valid MSE: 		 0.019315
Valid R2 (x100): 	 1.4096

Test MSE: 		 0.015836
Test R2 (x100): 	 0.50689


In [19]:
new_cols = add_rsi(data, 'xy_geom', twoweeks_agg_periods)
usecols.extend(new_cols)

model = Ridge(alpha=10)

selected_cols = greedy_add_strategy(model, data, selected_cols, new_cols,
                                    valid_ratio, test_ratio, droprows=7050)
validate_sklearn_model(model, data, selected_cols, valid_ratio, test_ratio, droprows=7050);

['xy_geom_rsi_6', 'xy_geom_rsi_60', 'xy_geom_rsi_360', 'xy_geom_rsi_720', 'xy_geom_rsi_1410', 'xy_geom_rsi_2820', 'xy_geom_rsi_7050', 'xy_geom_rsi_14100']
added xy_geom_rsi_360: r2: 1.4761
Data shapes:  (215660, 59) (68526, 59) (51394, 59)

Valid MSE: 		 0.019302
Valid R2 (x100): 	 1.4761

Test MSE: 		 0.015784
Test R2 (x100): 	 0.83259


In [21]:
new_cols = add_rsi(data, 'xy_garmonic', twoweeks_agg_periods)
usecols.extend(new_cols)

model = Ridge(alpha=10)

selected_cols = greedy_add_strategy(model, data, selected_cols, new_cols,
                                    valid_ratio, test_ratio, droprows=7050)
validate_sklearn_model(model, data, selected_cols, valid_ratio, test_ratio, droprows=7050);

['xy_garmonic_rsi_6', 'xy_garmonic_rsi_60', 'xy_garmonic_rsi_360', 'xy_garmonic_rsi_720', 'xy_garmonic_rsi_1410', 'xy_garmonic_rsi_2820', 'xy_garmonic_rsi_7050', 'xy_garmonic_rsi_14100']
added xy_garmonic_rsi_360: r2: 1.6273
Data shapes:  (215660, 67) (68526, 67) (51394, 67)

Valid MSE: 		 0.019272
Valid R2 (x100): 	 1.6273

Test MSE: 		 0.015846
Test R2 (x100): 	 0.4434
